In [1]:
import numpy as np
from numpy import random
from prosstt import tree
from prosstt import simulation as sim
from prosstt import sim_utils as sut
import scipy as sp
rseed = 100 
random.seed(rseed)

import anndata as ad
# from scanpy import pp
# from scanpy.tl import diffmap

import pandas as pd

import os

In [2]:
N = 500
T = 10
P = 10000
top = [["A", "B"]]
branches = np.unique(np.array(top).flatten())
time = {b: T for b in branches}
G = 100
lineage = tree.Tree(topology=top, G=G, time=time, num_branches=len(branches), branch_points=0, modules=8)

# prior hyperparameter for h
a = min(1/lineage.modules,0.05)

# assign genes to programs to calculate relative average gene expression
# uMs, Ws, Hs = sim.simulate_lineage(lineage, a=a)
uMs, Ws, Hs = sim.simulate_lineage(lineage, a=a)
#uMs: rel_means (T*G); Ws: programs (T*modules); Hs:modules*G

/lustre/home/acct-clsyzs/clsyzs/.conda/envs/JLpython3/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [3]:
dat = pd.read_csv("geno.csv")
dat

,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9,snp10,...,snp9992,snp9993,snp9994,snp9995,snp9996,snp9997,snp9998,snp9999,snp10000,id
0,1,0,2,2,1,1,1,2,2,2,...,1,2,2,2,2,1,1,1,2,1
1,2,0,2,2,2,1,0,0,2,2,...,1,2,2,2,2,1,2,0,2,2
2,2,0,2,2,2,0,1,2,1,2,...,1,2,1,2,2,1,2,0,2,3
3,1,1,2,2,0,1,2,1,2,2,...,0,2,2,2,2,2,2,1,2,4
4,1,0,1,2,2,0,2,2,2,2,...,0,2,2,2,2,2,1,0,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,0,2,2,2,1,2,2,2,2,...,1,2,2,2,2,1,2,0,2,496
496,0,0,2,2,2,1,2,2,1,2,...,1,2,2,2,2,2,2,0,2,497
497,2,1,2,2,2,0,1,1,2,2,...,0,2,2,2,2,1,2,2,2,498
498,1,0,2,2,1,0,2,2,1,2,...,2,2,1,2,2,1,2,0,2,499


In [4]:
# simulate eqtl effect
ntr = list(np.random.randint(15,25,G))
def getlambda(i):
    tr = np.random.choice(np.arange(1,P+1),ntr[i-1],replace=False)  #np.arange(ntr*(i-1)+1, ntr*i+1)
    beta = np.random.gamma(shape=3.6, scale=1/12, size=ntr[i-1])
    lambda_ = np.dot(dat.iloc[:, tr-1], beta)
    return tr,lambda_

result_list = list(map(getlambda, range(1, G+1)))
tr = [result[0] for result in result_list]
lambda_ = np.array([result[1] for result in result_list]) #G*N

In [5]:
# true eqtls
arr = np.zeros((G,P), dtype=int)
for i, sublist in enumerate(tr):
    arr[i, sublist-1] = 1
dfiv = pd.DataFrame(np.transpose(arr))
dfiv.columns = ["gene" + str(g) for g in range(1, G+1)]
dfiv['snp'] = ["snp" + str(p) for p in range(1, P+1)]
dfiv

,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,gene10,...,gene92,gene93,gene94,gene95,gene96,gene97,gene98,gene99,gene100,snp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp9996
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp9997
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp9998
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,snp9999


In [6]:
# scale relative average expression to absolute values
# base expression
gene_scale = sut.simulate_base_gene_exp(lineage, uMs)  #G*1

time-invariant SNP-gene

In [7]:
os.chdir("stable_eqtl")

In [8]:
lineage_list = []
dfmu_list = []

In [9]:
import copy
for i in range(N):
    lineage1 = copy.deepcopy(lineage)
#     gene_scale1 = lambda_[:,i]*gene_scale+gene_scale
#     lineage1.add_genes(uMs, gene_scale1)
    lineage1.add_genes(uMs, gene_scale)
    for j in list(lineage1.means.keys()):
        ts = np.arange(lineage1.branch_times()[j][0],lineage1.branch_times()[j][1]+1)
        # lambda_1 = np.outer(np.sqrt(2)*np.abs(np.sin(ts)),lambda_[:,i]) #time-varying
        lambda_1 = np.outer(ts*0+1,lambda_[:,i]) #time-invariant
        lineage1.means[j] = lineage1.means[j]+lineage1.means[j]*lambda_1
    lineage_list.append(lineage1)
    dfmu1 = pd.DataFrame(np.concatenate(list(lineage1.means.values())))
    dfmu1.columns = ["gene" + str(g) for g in range(1, G+1)]
    dfmu1["pseudotime"] = np.array([np.arange(i,j+1) for i,j in lineage1.branch_times().values()]).flatten()
    dfmu1['state'] = np.array([[i]*j for i,j in lineage1.time.items()]).flatten()
    dfmu1['id'] = i+1
    dfmu_list.append(dfmu1)

In [10]:
dfmu = pd.concat(dfmu_list)

In [11]:
dfmu

,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,gene10,...,gene94,gene95,gene96,gene97,gene98,gene99,gene100,pseudotime,state,id
0,124.613596,11.337746,2.236140,12.348499,37.607645,32.064244,2.880070,79.970705,10.889064,8.448009,...,2.052631,6.546275,7.438673,19.044293,2.106621,25.504369,5.898744,0,A,1
1,124.197781,11.244837,1.863996,12.168457,37.614177,31.424464,2.882845,89.471660,10.449944,7.772179,...,2.373599,6.513108,7.193719,18.948317,2.110581,24.637173,5.962687,1,A,1
2,124.382649,11.156359,1.607831,12.391546,37.619720,30.916854,2.875879,68.218241,9.239699,7.238980,...,2.614834,6.550926,6.360082,19.247631,2.114250,22.043163,6.012407,2,A,1
3,124.661663,11.097648,1.420547,12.639051,37.624967,30.788788,2.866109,46.682721,8.339103,6.801720,...,2.941287,6.595251,5.744641,19.174839,2.117856,20.102256,6.075347,3,A,1
4,124.931853,11.054899,1.287643,12.893107,37.629305,30.053908,2.851787,26.828588,7.576346,5.859015,...,2.892537,6.641427,5.345139,18.774695,2.116835,18.831632,6.080457,4,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,119.110955,11.550149,0.629256,13.119732,29.657568,20.232579,2.785411,35.090413,16.183870,12.806463,...,1.835051,5.796943,12.658721,23.442431,1.972740,29.910424,5.314439,15,B,500
16,120.541809,12.035872,0.686518,13.822044,29.665682,20.039738,2.785922,34.801619,21.498072,14.312622,...,1.617532,5.903149,16.997051,24.005342,1.986595,38.714035,5.262362,16,B,500
17,121.840594,12.539535,0.712819,14.439914,29.665057,18.989123,2.787289,35.829435,28.946508,15.608381,...,1.239057,5.995290,23.346878,24.292456,1.988068,51.061008,5.154662,17,B,500
18,122.332774,12.769938,0.803525,14.649983,29.648747,18.414529,2.782338,29.345978,33.257634,16.846255,...,0.922137,6.022127,27.021607,24.633256,1.969000,58.204877,5.042316,18,B,500


In [12]:
dfmu.to_csv("true_sc.csv",index=0)
dfiv.to_csv("dfiv.csv",index=0)

In [13]:
# sampling varaince hyperparameters
alpha = np.exp(random.normal(loc=np.log(0.2), scale=np.log(1.5), size=lineage.G))
beta = np.exp(random.normal(loc=np.log(1), scale=np.log(1.5), size=lineage.G)) + 1

In [14]:
df_list = []
for i in range(N):
    lineage1 = lineage_list[i]
    
    n = np.random.randint(8,12)
    X, labs, brns, scalings = sim.sample_whole_tree(lineage1, n, alpha=alpha, beta=beta)
    
    df1 = pd.DataFrame(X)
    df1.columns = list(map(lambda x: "gene"+str(x),range(1,G+1)))
    df1['pseudotime'] = labs
    df1['state'] = brns
    df1 = df1.sort_values(["pseudotime"])
    df1['id'] = i+1
    df_list.append(df1)

In [15]:
df = pd.concat(df_list)

In [16]:
df['cellid'] = df['id'].astype(str) + '_' + (df.groupby('id').cumcount()).astype(str)

In [17]:
df

,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,gene10,...,gene95,gene96,gene97,gene98,gene99,gene100,pseudotime,state,id,cellid
0,219,46,1,5,168,112,12,64,5,14,...,16,7,25,3,40,2,0,A,1,1_0
1,58,11,1,5,27,4,1,22,3,7,...,1,0,8,1,12,2,0,A,1,1_1
2,69,26,3,5,59,52,5,19,6,13,...,11,11,46,2,27,10,0,A,1,1_2
3,404,48,2,35,92,94,5,199,31,21,...,20,25,87,2,88,11,0,A,1,1_3
4,85,2,1,15,45,18,1,92,6,5,...,8,10,9,0,36,4,0,A,1,1_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,64,5,1,12,45,11,2,11,14,8,...,2,36,26,1,45,5,19,B,500,500_215
215,110,4,3,18,32,28,0,17,20,18,...,5,14,37,0,73,3,19,B,500,500_216
216,122,9,1,15,13,54,5,31,21,22,...,11,28,34,1,85,6,19,B,500,500_217
217,282,29,4,59,68,53,8,54,129,53,...,11,57,51,2,179,7,19,B,500,500_218


In [18]:
df.to_csv("complete_sample/sim_sc.csv",index=0)

In [19]:
df_list = []
for i in range(N):
    lineage1 = lineage_list[i]
    
    maxT = lineage1.get_max_time()
    n = np.random.randint(10*maxT, 15*maxT)

    series_points = list(np.random.choice(np.arange(0,maxT),5,replace=False))
    point_std = list(np.random.uniform(0.5,2,5))
    X, labs, brns, scalings = sim.sample_pseudotime_series(lineage1, cells=n,
                                                           series_points=series_points,
                                                           point_std=point_std,
                                                           alpha=alpha, beta=beta)

    # X = (X.transpose() / scalings).transpose()
    df1 = pd.DataFrame(X)
    df1.columns = list(map(lambda x: "gene"+str(x),range(1,G+1)))
    df1['pseudotime'] = labs
    df1['state'] = brns
    df1 = df1.sort_values(["pseudotime"])
    df1['id'] = i+1
    df_list.append(df1)

In [20]:
df = pd.concat(df_list)
df['cellid'] = df['id'].astype(str) + '_' + (df.groupby('id').cumcount()).astype(str)
df

,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,gene10,...,gene95,gene96,gene97,gene98,gene99,gene100,pseudotime,state,id,cellid
143,244,30,2,23,61,89,1,96,20,20,...,13,32,43,12,66,0,0,A,1,1_0
33,254,55,11,31,93,49,2,257,21,9,...,5,18,42,2,33,13,0,A,1,1_1
124,202,19,0,25,48,47,3,168,5,11,...,7,13,22,5,27,3,0,A,1,1_2
121,139,5,0,8,62,19,4,26,14,13,...,2,3,12,2,30,4,0,A,1,1_3
150,83,9,5,12,19,11,2,49,13,6,...,12,1,12,4,15,6,0,A,1,1_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,251,18,0,12,8,39,2,3,5,23,...,6,23,10,1,26,2,14,B,500,500_225
102,48,11,0,7,17,22,1,15,9,8,...,2,14,6,1,8,5,14,B,500,500_226
47,518,13,1,7,28,18,2,98,41,26,...,20,25,23,0,61,5,15,B,500,500_227
69,34,3,0,9,5,3,0,3,6,3,...,0,3,1,3,13,4,15,B,500,500_228


In [21]:
df.to_csv("mixed_sample/sim_sc.csv",index=0)

time-varying SNP-gene

In [22]:
os.chdir("../dyn_eqtl")

In [23]:
lineage_list = []
dfmu_list = []
import copy
for i in range(N):
    lineage1 = copy.deepcopy(lineage)
#     gene_scale1 = lambda_[:,i]*gene_scale+gene_scale
#     lineage1.add_genes(uMs, gene_scale1)
    lineage1.add_genes(uMs, gene_scale)
    for j in list(lineage1.means.keys()):
        ts = np.arange(lineage1.branch_times()[j][0],lineage1.branch_times()[j][1]+1)
        lambda_1 = np.outer(np.sqrt(2)*np.abs(np.sin(ts)),lambda_[:,i]) #time-varying
        # lambda_1 = np.outer(ts*0+1,lambda_[:,i]) #time-invariant
        lineage1.means[j] = lineage1.means[j]+lineage1.means[j]*lambda_1
    lineage_list.append(lineage1)
    dfmu1 = pd.DataFrame(np.concatenate(list(lineage1.means.values())))
    dfmu1.columns = ["gene" + str(g) for g in range(1, G+1)]
    dfmu1["pseudotime"] = np.array([np.arange(i,j+1) for i,j in lineage1.branch_times().values()]).flatten()
    dfmu1['state'] = np.array([[i]*j for i,j in lineage1.time.items()]).flatten()
    dfmu1['id'] = i+1
    dfmu_list.append(dfmu1)
    
dfmu = pd.concat(dfmu_list)

In [24]:
dfmu.to_csv("true_sc.csv",index=0)
dfiv.to_csv("dfiv.csv",index=0)

In [25]:
# sampling varaince hyperparameters
alpha = np.exp(random.normal(loc=np.log(0.2), scale=np.log(1.5), size=lineage.G))
beta = np.exp(random.normal(loc=np.log(1), scale=np.log(1.5), size=lineage.G)) + 1

In [26]:
df_list = []
for i in range(N):
    lineage1 = lineage_list[i]
    
    n = np.random.randint(8,12)
    X, labs, brns, scalings = sim.sample_whole_tree(lineage1, n, alpha=alpha, beta=beta)
    
    df1 = pd.DataFrame(X)
    df1.columns = list(map(lambda x: "gene"+str(x),range(1,G+1)))
    df1['pseudotime'] = labs
    df1['state'] = brns
    df1 = df1.sort_values(["pseudotime"])
    df1['id'] = i+1
    df_list.append(df1)

df = pd.concat(df_list)
df['cellid'] = df['id'].astype(str) + '_' + (df.groupby('id').cumcount()).astype(str)
df.to_csv("complete_sample/sim_sc.csv",index=0)

In [27]:
df_list = []
for i in range(N):
    lineage1 = lineage_list[i]
    
    maxT = lineage1.get_max_time()
    n = np.random.randint(10*maxT, 15*maxT)

    series_points = list(np.random.choice(np.arange(0,maxT),5,replace=False))
    point_std = list(np.random.uniform(0.5,2,5))
    X, labs, brns, scalings = sim.sample_pseudotime_series(lineage1, cells=n,
                                                           series_points=series_points,
                                                           point_std=point_std,
                                                           alpha=alpha, beta=beta)

    # X = (X.transpose() / scalings).transpose()
    df1 = pd.DataFrame(X)
    df1.columns = list(map(lambda x: "gene"+str(x),range(1,G+1)))
    df1['pseudotime'] = labs
    df1['state'] = brns
    df1 = df1.sort_values(["pseudotime"])
    df1['id'] = i+1
    df_list.append(df1)

In [28]:
df = pd.concat(df_list)
df['cellid'] = df['id'].astype(str) + '_' + (df.groupby('id').cumcount()).astype(str)
df.to_csv("mixed_sample/sim_sc.csv",index=0)